Companion Jupyter Notebook to the article [**A Quick Introduction to Google Earth Engine**](https://towardsdatascience.com/a-quick-introduction-to-google-earth-engine-c6a608c5febe) published in Towards Data Science.

## Setting your toolbox

In [ ]:
!pip install earthengine-api
!pip install folium 
!pip install geehydro

In [1]:
import ee 
import folium
import geehydro

from datetime import datetime as dt
from IPython.display import Image

## Initialize the connection to the server

In [2]:
ee.Initialize()

## Select a region in the world

In [3]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[-4.06738, -52.034], zoom_start=10)
Ituna_map

## Setting the Area of Interest (AOI)

In [4]:
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])

## The Landsat 8 Collection

In [5]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# choose dates
landsat = landsat.filterDate('2019-07-01','2019-12-01')
# filter area
landsat_AOI = landsat.filterBounds(Ituna_AOI)

## A bit of meta-data

In [6]:
landsat_AOI.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'LANDSAT/LC08/C01/T1_SR',
 'version': 1582201046453568,
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'visualization_1_bands': 'B5,B4,B3',
  'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png',
  'visualization_1_max': '30000.0',
  'description': '<p>This dataset is the atmospherically corrected\nsurface reflectance from  the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and two thermal infrared (TIR) bands processed to orthorectified\nbrightness temperature</p><p>These data have been atmospherically corrected using\n<a href="https://landsat.usgs.gov/sites/default/files/documents/lasrc_product_guide.pdf">LaSRC</a>\nand includes a cloud, shadow, water and snow mask produced using\n<a href="https://landsat.usgs.gov/what-cfmask">CFMASK</a>,\nas well

In [7]:
# total number of images.
print('Total number:', landsat_AOI.size().getInfo())

Total number: 9


In [8]:
# the names of each Landsat 8 band
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

# Choosing an Image

In [9]:
# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())

In [10]:
# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

Cloud Cover (%): 0


In [11]:
# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))
time = date.getInfo()['value']/1000.
dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

'2019-08-11 13:36:22'

## Visualizing the Satellite imagery

In [12]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}
             
Image(url = least_cloudy.getThumbUrl(parameters))

## Normalized Difference Vegetation Index (NDVI)

In [13]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])

In [14]:
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Ituna_AOI}

Ituna_map.addLayer(ndvi, ndvi_parameters)
Ituna_map